# Extract frames from videos

Pull frames from video files at specified intervals for analysis, thumbnails, or training data.

## Problem

You have video files and need to extract frames for:

- Object detection on video content
- Creating thumbnails or previews
- Building training datasets
- Scene analysis and classification

| Video | Duration | Frames at 1 FPS |
|-------|----------|-----------------|
| interview.mp4 | 30 min | 1,800 frames |
| product_demo.mp4 | 5 min | 300 frames |
| surveillance.mp4 | 1 hour | 3,600 frames |

## Solution

**What's in this recipe:**
- Extract frames at a fixed rate (FPS)
- Extract a specific number of frames
- Extract only keyframes for efficiency

You create a view with a FrameIterator that automatically extracts frames from each video. New videos are processed without extra code.

### Setup

In [ ]:
%pip install -qU pixeltable

In [ ]:
import pixeltable as pxt
from pixeltable.iterators import FrameIterator

### Load videos

In [ ]:
# Create a fresh directory
pxt.drop_dir('video_demo', force=True)
pxt.create_dir('video_demo')

In [ ]:
# Create table for videos
videos = pxt.create_table('video_demo.videos', {'video': pxt.Video})

In [ ]:
# Insert a sample video
videos.insert([
    {'video': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/bangkok.mp4'}
])

### Extract frames at fixed rate

Create a view that extracts 1 frame per second:

In [ ]:
# Extract 1 frame per second
frames = pxt.create_view(
    'video_demo.frames',
    videos,
    iterator=FrameIterator.create(
        video=videos.video,
        fps=1.0  # 1 frame per second
    )
)

In [ ]:
# View extracted frames
frames.select(frames.frame, frames.pos).head(3)

### Extract keyframes only

For faster processing, extract only keyframes (I-frames):

In [ ]:
# Extract only keyframes (much faster for long videos)
keyframes = pxt.create_view(
    'video_demo.keyframes',
    videos,
    iterator=FrameIterator.create(
        video=videos.video,
        keyframes_only=True
    )
)
keyframes.select(keyframes.frame).head(3)

## Explanation

**Extraction options:**

| Parameter | Description |
|-----------|-------------|
| `fps=1.0` | Extract at 1 frame per second |
| `fps=0.5` | Extract every 2 seconds |
| `num_frames=10` | Extract exactly 10 evenly-spaced frames |
| `keyframes_only=True` | Extract only I-frames (fastest) |

Only one of `fps`, `num_frames`, or `keyframes_only` can be specified.

**When to use keyframes:**

- Quick video scanning and thumbnails
- Initial content classification
- Processing very long videos

**Frame metadata:**

Each frame includes:
- `frame`: The extracted image
- `pos`: Frame position in the video
- `pts`: Presentation timestamp

## See also

- [Iterators documentation](https://docs.pixeltable.com/platform/iterators)
- [Analyze images in batch](https://docs.pixeltable.com/howto/cookbooks/images/vision-batch-analysis)